In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [ ]:
import dogs_cats_config as config
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreProcessor
from loader_util.preprocessing import PatchPreprocessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.callbacks import TrainingMonitor
from loader_util.nn.conv import AlexNet
##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [ ]:
# construct the trian image generator
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# load the RGB means
means = json.loads(open(config.DATASET_MEAN_PATH).read())

# init the image processors
sp = SimplePreProcessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# init the train and validation dataset generators
train_gen = HDF5DatasetGenerator(db_path=config.TRAIN_HDF5_PATH,
                                 batch_size=128,
                                 aug=aug,
                                 preprocessors=[pp, mp, iap],
                                 classes=2)

val_gen = HDF5DatasetGenerator(db_path=config.VAL_HDF5_PATH,
                               batch_size=128,
                               preprocessors=[sp, mp, iap],
                               classes=2)

In [ ]:
print(f"[INFO] compiling model......")
opt = Adam(learning_rate=0.001)
model = AlexNet.build(width=227, height=227, depth=3,
                      classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# construct the path of call backs
path = os.path.sep.join([config.OUTPUT_DIR, f"{os.getpid()}.png"])
callbacks = [TrainingMonitor(path)]


In [ ]:
# train the network
H = model.fit_generator(train_gen.generator(),
                        steps_per_epoch=train_gen.num_images // 128,
                        validation_data=val_gen.generator(),
                        validation_steps=val_gen.num_images // 128,
                        epochs=75,
                        max_queue_size=10,
                        callbacks=callbacks,
                        verbose=1)

In [ ]:
print(f"[INFO] serializing model......")
model.save(config.MODEL_PATH, overwrite=True)

# close the datasets
train_gen.close()
val_gen.close()